# Convert a pandas dataframe to geojson for web-mapping

In [1]:
import pandas as pd, json as json, urllib

First download data from the city of Berkeley's API. You can use Socrata's $limit parameter to specify how many rows to grab (otherwise the default is 1,000 rows of data): https://dev.socrata.com/docs/paging.html

In [2]:
# API endpoint for city of Berkeley's 311 calls
endpoint_url = 'https://data.cityofberkeley.info/resource/k489-uv4i.json?$limit=5000'

In [3]:
# open a connection to the URL
connection = urllib.urlopen(endpoint_url)

# download the results
results = connection.read()

# parse the string into a Python data structure
data = json.loads(results)

Next, turn the json data into a dataframe and clean it up a bit: drop unnecessary columns and any rows that lack lat-long data. We want to make our json file as small as possible (prefer under 5 mb) so that it can be loaded over the Internet to anyone viewing your map, without taking forever to download a huge file.

In [4]:
# turn the json data into a dataframe and see how many rows and what columns we have
df = pd.DataFrame(data)

print 'We have {} rows'.format(len(df))
str(df.columns.tolist())

We have 5000 rows


"[u'apn', u'city', u'indbdate', u'issue_description', u'issue_type', u'latitude', u'location', u'longitude', u'neighborhood_district', u'object_type', u'secondary_issue_type', u'state', u'street_address', u'ticket_closed_date_time', u'ticket_created_date_time', u'ticket_id', u'ticket_status']"

In [5]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['street_address'] = df['street_address'].str.title()

In [6]:
# we don't need all those columns - only keep useful ones
cols = ['issue_description', 'issue_type', 'latitude', 'longitude', 'street_address', 'ticket_status']
df_subset = df[cols]

In [7]:
# drop any rows that lack lat/long data
df_geo = df_subset.dropna(subset=['latitude', 'longitude'], axis=0, inplace=False)

print 'We have {} geotagged rows'.format(len(df_geo))
df_geo.tail()

We have 2438 geotagged rows


,issue_description,issue_type,latitude,longitude,street_address,ticket_status
4994,Residential Service Start,Refuse and Recycling,37.894299,-122.258158,1044 Creston Rd,Closed
4995,Commercial Reminder,Refuse and Recycling,37.874195,-122.268368,1849 Shattuck Ave,Closed
4996,Illegal Dumping - City Property,"Streets, Utilities, and Transportation",37.857278,-122.264605,2147 Russell St,Closed
4998,Residential Missed Pickup Integration,Refuse and Recycling,37.861356,-122.291607,2429 Ninth St D,Closed
4999,Residential Cart Size Decrease,Refuse and Recycling,37.872453,-122.284992,1374 Delaware St,Closed


In [53]:
import random
import numpy as np

df_geo.loc[:, 'ix1'] = pd.Series(np.random.randn(len(df_geo.index)), index=df_geo.index)
df_geo.loc[:, 'ix2'] = pd.Series(np.random.randn(len(df_geo.index)), index=df_geo.index)
df_geo.head(11)

,issue_description,issue_type,latitude,longitude,street_address,ticket_status,ix1,ix2
0,Commercial Cart Size Increase,Refuse and Recycling,37.854654,-122.248474,3084 Claremont Ave,Closed,-0.871942,-0.343552
1,Residential Site Inspection,Refuse and Recycling,37.862779,-122.257938,2612 Regent St,Closed,0.537646,0.832272
2,Residential Lost or Stolen Cart,Refuse and Recycling,37.857057,-122.273208,1747 Oregon St,Closed,-0.562705,0.661352
3,Residential Bulky Pickup,Refuse and Recycling,37.855207,-122.274057,2911 King St,Open,1.201637,-0.096207
5,Miscellaneous Service Request,General Questions/information,37.878108,-122.284532,1474 Rose St,Closed,-1.118401,0.535427
8,Residential Bulky Pickup,Refuse and Recycling,37.851674,-122.275183,1626 Prince St,Closed,1.181152,-0.201018
10,Commercial Reminder,Refuse and Recycling,37.891301,-122.284619,1597 Solano Ave,Closed,0.886818,-0.816203
12,Residential Reminder,Refuse and Recycling,37.868096,-122.290493,1124 Addison St,Closed,0.364259,0.469727
13,Commercial Missed Pickup,Refuse and Recycling,37.862315,-122.285405,1315 Dwight Way,Closed,1.122068,-0.826390
18,Cart Repair,Refuse and Recycling,37.878588,-122.279762,1624 Buena Ave,Closed,-1.893104,0.695461


In [54]:
# what is the distribution of issue types?
df_geo['issue_type'].value_counts()

Refuse and Recycling                            1823
General Questions/information                    236
Streets, Utilities, and Transportation           227
Parks, Trees and Vegetation                       51
Business License                                  29
Environmental Services and Programs               27
Facilities, Electrical & Property Management      19
Traffic and Transportation                        11
Graffiti and Vandalism                             9
Equipment Maintenance                              4
Other Account Services and Billing                 2
Name: issue_type, dtype: int64

Finally, convert each row in the dataframe to a geojson-formatted feature and save the result as a file. The format is pretty simple and you can see it here: http://geojson.org/

In [55]:
# create a new python dict to contain our geojson data, using geojson format
geojson = {'type':'FeatureCollection', 'features':[]}

In [56]:
# loop through each row in the dataframe and convert each row to geojson format
for _, row in df_geo.iterrows():
    feature = {'type':'Feature',
               'geometry':{
                   'type':'Point',
                   'coordinates':[row['longitude'],row['latitude']]},
               'properties': {
                   'street_address':row['street_address'],
                   'issue_description':row['issue_description'], 
                   'issue_type':row['issue_type'],
                   'ticket_status':row['ticket_status'],
                   'ix1':row['ix1'],
                   'ix2':row['ix2']}}
    
    # add this feature (aka, converted dataframe row) to the list of features inside our dict
    geojson['features'].append(feature)

In [57]:
# save the geojson result to a file
output_filename = 'dataset_ix.js'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2)  
    
# how many features did we save to the geojson file?
print '{} geotagged features saved to file'.format(len(geojson['features']))

2438 geotagged features saved to file


Now just load that dataset.js file with leaflet to map it. See berkeley-311-map.html for an example of creating the map, and see sample-blog-post.html for an example of how to display this map inside another web page.